In [1]:
import urllib.request
import re
import sys
import codecs
import json
from datetime import datetime
import os
import glob
import utils
import time
import getopt
import ssl
import pandas as pd 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']#设置字体以便支持中文
plt.rcParams['savefig.dpi'] = 300 #图片像素
plt.rcParams['figure.dpi'] = 300 #分辨率
plt.rcParams['axes.unicode_minus']=False  #set for displaying `-

In [3]:
# 设置参数

g_token = "c68bb70c102d863b3d96bc5b8901ff7689dfcba68f040b4fb3d56e9b2f8dbab8"
g_app_key = "e70d4c831ca1c79442f7ef7a68aeb9e3"
g_user_id = "5c16ff1d6e249c62d057987f"
g_board_id = "5ac3159ed146b0793ff30206" # ThunderDB
# g_board_id = "5b640e00c8b518181d80fa46" # Rating system
g_crontab_style = False

workload_pattern = u'[(（]\s*(\d+(?:\.\d+)?)\s*(pt|PT|pT|Pt)\s*[)）]' # 正则表达式
requirement_pattern = u'[\[【［]\s*(.*)\s*[】］\]]\s*' 

In [4]:
def basic_replace(url): # 替换 url 等主要参数
    url = url.replace("_APP_KEY_", g_app_key)
    url = url.replace("_TOKEN_", g_token)
    return url

def do_request(url): 
    context = ssl._create_unverified_context()
    request = urllib.request.Request(url) # urllib 网页抓取的库
    content = None
    flag = False

    for i in range(3):  # retry when network access failed 等于说这里是尝试了三遍
        try:
            response = urllib.request.urlopen(request, context= context) # 会验证一次 SSL 证书, 当目标使用的是自签名的证书时就会报 SSL 错误
            content = response.read().decode()
            flag = True # flag 应该是用来判断 try 是否执行成功
            break
        except urllib.request.HTTPError as e: # 给出报错信息
            print("http error: " + str(e))
        except Exception as e:
            print("error: " + str(e))

    if not flag:
        sys.exit(-1) # 没有 flag 说明报错了，然后退出程序

    if content is None:
        return []
    else:
        return json.loads(content)

In [5]:
def get_card_name_by_pattern(card_name, pattern):
    return re.findall(pattern, card_name, re.S | re.U) # 搜索字符串，以列表类型返回全部能匹配的子串

def fetch_board_by_user():
    url = 'https://api.trello.com/1/members/_USERID_/boards?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_USERID_", g_user_id)
    body = do_request(url)
    board_info = {}

    for item in body:
        board_info[item['shortLink']] = item['name']

    return board_info # 拿到 board 的 name 和 shortlink 意味着可以放进 url 直接访问页面

def insertStr(str_name): 
    
    if "[" in str_name: # 沙雕了。。。“[]”是意味着两个连在一起的
        str_list = list(str_name) # 字符数，可以利用这个在某个位置插入字
        str_pos = str_list.index('[') # 找到[ 的位置
        str_list.insert(str_pos, '\\') # 插入要插入的字符
        str_2 = "".join(str_list) # 将 list 转为 str
    
    else:
        str_2 = str_name
    
    return str_2

def get_card_name_by_pattern(card_name, pattern):
    return re.findall(pattern, card_name, re.S | re.U)

def fetch_list_id_by_board(list_pattern): # list_pattern 作为参数要输入 list name 
    url = 'https://api.trello.com/1/boards/_BOARDID_/lists?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url) # 返回那个 board 里所有的 list 信息
    list_info = []
    list_name = insertStr(list_pattern) # 识别 "[]" 
    
    for item in body:
        
        if re.search(list_name, item["name"], re.I) is not None: # re.search 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象
            list_info.append({'id': item['id'], 'name': item['name']}) 
            

    return list_info # 返回 list name 和 list_id

def fetch_members_by_board(): # 拿到所有成员的名称和 id
    url = 'https://api.trello.com/1/boards/_BOARDID_/members?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url)
    all_members_info = {}

    for item in body:
        all_members_info[item['id']] = item['fullName']

    return all_members_info # 返回 id 和 member name

def fetch_cards_by_list_id(list_id): #  可以通过上面的 fetch_list_id_by_board 可以获取 list_id
    url = 'https://api.trello.com/1/lists/_LISTID_/cards?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_LISTID_", list_id) 
    body = do_request(url)

    return body # 用 list_id 获取 cards 的所有信息


In [140]:
def get_cards_info(list_id, board_members):

    available_cards_info = []
    all_cards_info = fetch_cards_by_list_id(list_id)

    #   项目名称 项目种类 负责人 项目耗时 项目周 描述

    for item in all_cards_info:
        card_info = { # 设置很多空的 dict，然后依据后面的判断填入一些参数
            'card_type': None,
            'card_name': item['name'],
            'card_id': item['id'],
            'member_id': None,
            'member_name': None,
            'plan_hours': 0,
            'actual_hours': 0, # 现在好像没有分 actual hours 和 plan 
            "description" : item["desc"]
        }

        
        if len(item["name"].split("【")) > 1:
            card_info["card_type"] = item["name"].split("【")[1].split("】")[0]
            
            
        idMembers = item['idMembers'] # 让 cards_info get 到 member 的信息
        if len(idMembers) > 0 and idMembers[0] is not None:
            member_id = idMembers[0]
            card_info['member_id'] = member_id
            card_info['member_name'] = board_members[member_id] # 这里不是报错是和 fetch_members_by_board 一起获得

        labels = item['labels']

        if len(labels) > 0 and labels[0] is not None: # 处理卡片的 labels
            label = labels[0]
            card_info['label_id'] = label['id']
            card_info['label_name'] = label['name']

        card_hours = get_card_name_by_pattern(item['name'], workload_pattern)

        if len(card_hours) == 2:
            card_info['actual_hours'] = float(card_hours[1][0]) * 2
            card_info['plan_hours'] = float(card_hours[0][0]) * 2
        else:
            card_info['actual_hours'] = float(card_hours[0][0]) * 2


        available_cards_info.append(card_info)

    return available_cards_info

In [87]:
def period_list(list_name): # 通过输入单个的 list_name，然后统计从那个时候到现在的个人工作时间
    list_id = fetch_list_id_by_board(list_name)
    id_list = []
    url = 'https://api.trello.com/1/boards/_BOARDID_/lists?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url)
    
    for i in body:
        id_list.append(i["id"])
        
    for item in body: # 这里没有 item 的异常处理
        if item["id"] == list_id[0]["id"]: 
            id_pos = id_list.index(item["id"]) # 写漏了 id_list 从哪儿来

    return id_list[3:id_pos+1]

In [125]:
period_list("Done[20190215]")

['5d06eef0b6e95e05d8bea068',
 '5cfdb2a0d4ebe10c61bf15b7',
 '5cf3d8f30b906c18a1591a98',
 '5cec905c56e9c039cc879a01',
 '5ce1f23a7103e93bbfb5fa02',
 '5cd8c0cba986e325375046ba',
 '5cce46a2384b4b3dcbb12531',
 '5cc652288dda046d54b00cbc',
 '5cbc75920f2ce20a9e39fed8',
 '5cb482db77cdf785b9b4777f',
 '5caa070d0f951d86f68f67b4',
 '5ca0c84744259a35c842e3b3',
 '5c983270b11ed084aa19d1d0',
 '5c8f0efb6245917192960b79',
 '5c85bed81d73ea51b24bc340',
 '5c7c7973876d6309b53d5c76',
 '5c73a25bb856025094b60989',
 '5c6a1792e64bec80de03adf8',
 '5c605b17a623733945ae9498']

In [141]:
get_cards_info("5ce1f23a7103e93bbfb5fa02", board_members)
# available_cards_info = []
# all_cards_info = fetch_cards_by_list_id("5ce1f23a7103e93bbfb5fa02")

[{'card_type': '招聘',
  'card_name': '【招聘】前端招聘 (4pt)',
  'card_id': '5ce10e7332ccf822175ecc12',
  'member_id': '5ad8c41afae1085dac4eb40b',
  'member_name': 'chenxi',
  'plan_hours': 0,
  'actual_hours': 8.0,
  'description': 'https://www.yuque.com/covenantsql/vmodzd/qtwkiw\n题目 & 答案\n\nhttps://www.yuque.com/covenantsql/vmodzd/bqpkyw\n反馈\n'},
 {'card_type': 'CQL8',
  'card_name': '【CQL8】onboarding 开发 (8pt)',
  'card_id': '5ce10ef9cf9e5a6025397d0a',
  'member_id': '5ad8c41afae1085dac4eb40b',
  'member_name': 'chenxi',
  'plan_hours': 0,
  'actual_hours': 16.0,
  'description': 'UI 切面已经 OK，需要联调'},
 {'card_type': 'CQL8',
  'card_name': '【CQL8】用户 onboarding 流程 & 线框设计 (4pt)',
  'card_id': '5ce10a7aa5fdd9333fd16a81',
  'member_id': '5ad8c41afae1085dac4eb40b',
  'member_name': 'chenxi',
  'plan_hours': 0,
  'actual_hours': 8.0,
  'description': '产出线框图\nhttps://www.yuque.com/covenantsql/xsx66o/awbaih'},
 {'card_type': 'CQL6',
  'card_name': '【CQL6】更新官网 https://covenantsql.io/#blog 的《CovenantSQL 从

In [145]:
board_members = fetch_members_by_board()


df_list = []

for item in period_list("Done[20190215]")[7:]:
    
    new_workinfo = get_cards_info(item, board_members)
    
    df_new = pd.DataFrame(new_workinfo)
    df_new.to_csv('CQL_{}.csv'.format(item))

In [202]:
files = os.listdir('/Users/lichenxi/venv/CovenantSQL/Trello Analyzer')

In [155]:
fn = files[0]
df = pd.read_csv(os.path.join('/Users/lichenxi/venv/CovenantSQL/Trello Analyzer', fn))

In [177]:
dfs = []
for fn in files:
    try:
        df = pd.read_csv(os.path.join('/Users/lichenxi/venv/CovenantSQL/Trello Analyzer', fn))
        df.set_index("Unnamed: 0", inplace=True)
        dfs.append(df)
    except Exception as e:
        print('Error loading: ', fn)
        print(e)

Error loading:  work_hours_chart.png
'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Error loading:  .ipynb_checkpoints
Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.
Error loading:  Trello_Analyzer_Original.ipynb
Error tokenizing data. C error: Expected 1 fields in line 4, saw 2

Error loading:  CQL_Statistic.ipynb
Error tokenizing data. C error: Expected 1 fields in line 4, saw 2

Error loading:  Trello_analyzer.ipynb
Error tokenizing data. C error: Expected 1 fields in line 4, saw 2



In [181]:
result = pd.concat(dfs,join='outer') 

/Users/lichenxi/venv/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [ ]:
result.to_csv("CQL_statistic.csv", encoding='utf_8_sig') # 防止中文乱码

In [ ]:
# CQL 等应该变成 columns 
# raw 依旧是时间
# 单元格中是所花的时间

In [ ]:
actual hours 是一个 list 包含了所有类型的工作时间

In [211]:
result.set_index("card_type")

,actual_hours,card_id,card_name,card_type,description,label_id,label_name,member_id,member_name,plan_hours
index,,,,,,,,,,
0,8.0,5cf77b9ab4d6ac7af78262e6,【CQL8】Task 相关设计 & 实现 (3pt)(4pt),CQL8,NaN,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,6.0
1,2.0,5cf7a6d73cac8c1b97344b2a,【CQL8】 Deploy 相关准备: stg-api， now.json（1pt）,CQL8,`stg-api.covenantsql.io/v3/admin/auth/authoriz...,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,0.0
2,4.0,5ceab78e44229a35094feb1e,【CQL8】项目管理流程设计 (2pt),CQL8,产出线框图\n\nhttps://www.yuque.com/covenantsql/xsx...,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,0.0
3,2.0,5cf63c740aeb5e560631f1f4,【CQL8】 Menu 上 Wallet 入口 (1pt),CQL8,NaN,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,0.0
4,2.0,5cf1067c0541170cb9f84e98,【CQL8】get PTC 入口 (1pt),CQL8,NaN,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,0.0
5,12.0,5cf106f77ca01c4e505edbe6,【CQL8】上传 wallet & set main wallet & 下载 & 删除 (3...,CQL8,NaN,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,6.0
6,2.0,5cf4f831a507d23952c18a2c,【tokenmetric】add lang params & deploy (1pt),tokenmetric,NaN,NaN,NaN,5ad8c41afae1085dac4eb40b,chenxi,0.0
7,8.0,5cf3cd0007b5805a847db356,【CQL10】Local File 结构设计 (4pt),CQL10,NaN,NaN,NaN,5293028f22779c60010068fc,刘志腾,0.0
8,4.0,5cf3cd38d6763b29e5fde904,【CQL10】SDK API 列表文档 (2pt),CQL10,NaN,NaN,NaN,5293028f22779c60010068fc,刘志腾,0.0


In [217]:
result["actual_hours"][2]

pandas.core.series.Series